In [234]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from data_processing import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from data_transformar import *
import lightgbm as lgb
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor, Pool

warnings.simplefilter("ignore")
%matplotlib inline
%load_ext autoreload
%autoreload 2

sns.set(style="darkgrid")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [235]:
data = pd.read_csv("./train.csv")
data = data.drop(columns=["Id"])

In [236]:
test_data = pd.read_csv("./test.csv")
test_data = test_data.drop(columns=["Id"])

In [237]:
data=data.drop(columns=['PoolQC', 'MiscFeature','Alley', 'Fence', 'FireplaceQu'], axis=1)
test_data=test_data.drop(columns=['PoolQC', 'MiscFeature','Alley', 'Fence', 'FireplaceQu'], axis=1)

In [238]:
y = data["SalePrice"]
X = data.select_dtypes(np.number) #data.drop(columns=["SalePrice"])

print(X.shape)

(1460, 37)


In [239]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [240]:
missing = X[X.columns[X.isna().any()]]
not_missing = X[X.columns[~X.isna().any()]]

not_missing.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,8450,7,5,2003,2003,706,0,150,856,...,0,61,0,0,0,0,0,2,2008,208500
1,20,9600,6,8,1976,1976,978,0,284,1262,...,298,0,0,0,0,0,0,5,2007,181500
2,60,11250,7,5,2001,2002,486,0,434,920,...,0,42,0,0,0,0,0,9,2008,223500
3,70,9550,7,5,1915,1970,216,0,540,756,...,0,35,272,0,0,0,0,2,2006,140000
4,60,14260,8,5,2000,2000,655,0,490,1145,...,192,84,0,0,0,0,0,12,2008,250000


In [243]:
columns = missing.columns

#single
column = columns[0]
#data in column with nans
data = missing[column]

#rows with no nans
not_nan_idx = np.where(~np.isnan(data))[0]
nan_idx = np.where(np.isnan(data))[0]

not_nan_data = np.asarray(data[not_nan_idx])
#rows of main table where no data is missing
not_nan_labels = not_missing.iloc[not_nan_idx]

######

nan_data = np.asarray(data[nan_idx])
#rows of main table where some data is nan
nan_labels = not_missing.iloc[nan_idx]

#np.unique(not_nan_data)
#not_nan_labels.isna().any()

In [250]:
#model = RandomForestRegressor()
X_train, X_test, y_train, y_test = train_test_split(not_nan_labels, not_nan_data, test_size=0.2, random_state=98987)
model = xgb.XGBRegressor(
                colsample_bytree=1.,
                eta=0.01,
                max_depth=4,
                min_child_weight=1.5,
                n_estimators=14400,                                                                  
                alpha=0.,
                reg_lambda=0.4,
                subsample=0.2)


model.fit(X_train, y_train)

XGBRegressor(alpha=0.0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, enable_categorical=False,
             eta=0.01, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.00999999978,
             max_delta_step=0, max_depth=4, min_child_weight=1.5, missing=nan,
             monotone_constraints='()', n_estimators=14400, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=0.4, scale_pos_weight=1, subsample=0.2,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [252]:
rmsle(model.predict(X_test), y_test)

0.5468168260171162

In [253]:
columns = missing.columns

predicted_fixed = []

for column in columns:
    #data in column with nans
    data = missing[column]

    #rows with no nans
    not_nan_idx = np.where(~np.isnan(data))[0]
    nan_idx = np.where(np.isnan(data))[0]

    not_nan_data = np.asarray(data[not_nan_idx])
    #rows of main table where no data is missing
    not_nan_labels = not_missing.iloc[not_nan_idx]

    ######

    nan_data = np.asarray(data[nan_idx])
    #rows of main table where some data is nan
    nan_labels = not_missing.iloc[nan_idx]

    model = xgb.XGBRegressor(
                colsample_bytree=1.,
                eta=0.01,
                max_depth=4,
                min_child_weight=1.5,
                n_estimators=14400,                                                                  
                alpha=0.,
                reg_lambda=0.4,
                subsample=0.2)

    model.fit(not_nan_labels, not_nan_data)

    #self test
    print(column, rmsle(model.predict(not_nan_labels), not_nan_data))

    predicted_fixed.append(model.predict(nan_labels))

LotFrontage
MasVnrArea
GarageYrBlt
